<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/NLP/2_Redes_Recurrentes/RNN.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Modelos de secuencias

Hasta ahora hemos trabajado con series de datos donde a cada entrada le corresponde una salida. Por ejemplo, a una imagen le corresponde una categoría. A una serie de indicadores biométricos le corresponde un diagnóstico médico.

En procesamiento de lenguajes naturales, nuestras salidas y nuestras entradas tienen una característica distinta. Veamos un ejemplo:

> *Usted tiene 16 años. Está prohibido vender alcohol a menores de 18 años. No puedo venderle esa botella.*

En el ejemplo anterior, tenemos tres afirmaciones, donde la última es un conclusión de las dos anteriores. En este sentido, cuando trabajamos con lenguajes tenemos el problema de lo próximo que se dice, depende de lo que se dijo antes. Es decir, estamos trabajando con secuencias temporales.

Peor aún, muchas veces la última salida, debe ser tratada como una nueva entrada. Piense en el ejemplo anterior, si usted vive en un país latinoamericano o europeo, al llegar a **menores de** intuía que la edad límite sería **18 años**. Eso es porque como ciudadano de su país, sabe que esa es la ley. Es decir. **18 años** podría haber sido una predicción, una salida de su red. Ademas, al haber predicho **18 años** ahora podemos concluir que **No puedo venderle esa botella**. Si la ley dijera que los menores de **14 años** pueden comprar alcohol, la segunda frase carecería de sentido. Es decir **18 años** es una predicción, una salida en un momento, pero luego se convierte en una entrada o un *feature* en otro.

Es por lo anterior que se dice que estos modelos son modelos autoregresivos: las salidas luego se convierten en entradas, como en un problema recursivo.

La naturaleza autogresiva de nuestro modelo hace que debamos considerar la calidad de nuestras predicciones. Volviendo al ejemplo anterior, si nuestra predicción hubiera sido **14 años** en lugar de **18 años**, la conclusión final de nuestra frase sería distinta a la que hemos obtenido. Pequeños errores en un nuestras predicciones pueden acumularse a lo largo del tiempo y generar resultados absurdos.



# Modelos markovianos y variables ocultas.

Volvamos de nuevo a nuestro ejemplo de oraciones 

> *Usted tiene 16 años. Está prohibido vender alcohol a menores de 18 años. No puedo venderle esa botella.*

Supongamos de nuevo que queremos predecir **18 años**. La cantidad de escritas hasta ese momento es 11. Luego de predecir **18 años**, la cantidad de palabras aumentó a 13. Es decir, conforme predecimos y agregamos información, nuestro modelo debe responder a la cantidad creciente de palabras o ejemplos.

Recordemos que todas estas herramientas nacieron de la estadística, por lo que nuestras predicciones se basaran en considerar la probabilidad de que diferentes palabras ocurran en simultaneo. Esto es verdaderamente un problema: mientas más palabras tenemos, menos probable es que vuelvan a ocurrir. Si ocurren infrecuentemente, necesitamos aumentar cada vez más la cantidad de ejemplos de nuestros datos. Esto puede ser un problema incluso para oraciones cortas. Una alternativa para paliar este problema es limitar la cantidad de palabra que miraremos hacia atras. 

Otra alternativa a esto es trabajar con variables ocultas. Las variables ocultas son cantidades que de alguna manera agrupan la información de todos los casos anteriores. Por ejemplo:

> *Usted es menor de edad. No puedo venderle esa botella.*

Hemos resumido toda la información de dos oraciones en una sola mucho más corta. 

De la misma manera que buscamos representaciones abstractas para palabras por medio de *tokens*, usaremos esos tokens para generar nuevas variables que resuman la información anterior. Es decir, generaremos una variable que de alguna manera tiene toda la información de **Usted es menor de edad**

Al trabajar con variables ocultas, esperamos reemplazar las todas las palabras anteriroes con el último valor de la variable oculta. Así, nuestro problema que antes veía 13 variables o tokens ahora ve uno solo. Esto nos permite simplificar nuestro modelo para trabajar con **modelos makovianos de primer orden**.

Sin entrar en mucho detalle, los modelos markovianos son el tipo de modelos autoregresivos como los que hemos descripto hasta ahora.

Esbozo de la noción de modelo de Markov

* Tenemos un estado (las últimas palabras escritas) a la cual llegamos a partir de un estado inicial bien definido
* Tenemos una historia de estados pasados que afecta  a estados futuros (cada palabra predicha o dentro de nuestro dataset)
* Hay una probabilidad asosciada a cada cambio de estado
* Queremos predecir cual será el próximo estado de un grupo finito de estados (la próxima palabra). 

Al trabajar con un modelo markoviano sobre variables ocultas, esperamos que la variable oculta resuma con tanta fidelidad los tokens pasados que solo necesitemos la variable oculta más reciente. Al necesitar solo la más reciente, se dice que es un modelo markoviano de primer orden (requiera solo una variable anterior). La razón por la que buscamos trabajar con modelos de primer orden es que son menos costosos computacionalmente.

En resumen nuestra propuesta para generar modelos de lenguaje consistira en lo siguiente:

1. Tomaremos texto de para crear nuestro *datset*
1. Transformaremos nuestro texto en algun tipo de representación simbólica (*tokens*)
2. De esta manera, nuestro modelo de lenguaje se convertirá en un problema de clasificiación: Dadas las palabras anteriores, ¿Cuál es la siguiente palabra? 
  * Decimos que es un problema de clasificación, porque cada una de nuestra palabras es una categoría.
3. Para crear nuestro modelo de lenguaje, usaremos variables ocultas en el contexto de un modelo markoviano.
  * La justificación para esto es que el lenguaje tiene caracterísitcas de un modelo markoviano. 



# Redes neuronales recurrentes

En la sección anterior intentamos argumentar que el lenguaje puede modelarse con un modelo markoviano. Además, propusimos trabajar con modelos markovinos con variables ocultas. La idea de trabajar con variables ocultas era poder trabajar con un modelo markoviano de primer orden. Queremos usar estos modelos de primer orden porque sabemos que nos permitiran ahorrar uso de memoria, así como disminuir el uso de recursos computacionales.

Nuestra propuesta para trabajar con variables ocultas, será trabajar con las unidades ocultas de un perceptrón multicapa 

![](http://d2l.ai/_images/mlp.svg)

$$\mathbf{O} = \mathbf{H} \mathbf{W} + \mathbf{b}$$
$$\mathbf{H} = \phi(\mathbf{X} \mathbf{W} + \mathbf{b})$$

Sin embargo, como trabajaremos con secuencias temporales, nuestra entrada al tiempo $t$, debe depender del tiempo $t-1$. Es decir, la próxima palabra debe depender de las palabras anteriores. En un perceptrón multicapa, esa dependecia temporal no está presente. Es por esto que debemos reestructurar nuestra capa para que permita generar modelos autoregresivos de secuencias.Dado queremos usar las variables ocultas como cantidades que resumen toda la información anterior, son estas cantidades las que tendran una dependencia temporal

$$\mathbf{O}_{t} = \mathbf{H}_{t} \mathbf{W}_{O} + \mathbf{b}$$
$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{X} + \mathbf{H}_{t-1} \mathbf{W}_{H}  + \mathbf{b}_h).$$

Notemos que $\mathbf{H}_t$ depende del valor anterior, $\mathbf{H}_{t-1}$ y del nuevo valor $\mathbf{X}_t$. Esta dependencia temporal es la que hace que nuestra nueva red neuronal sea una *red neuronal recurrente*. Recordemos si nuestro modelo está correctamente entrenado la salida $\mathbf{O}_t$ debe coincidir con el resultado correcto o *grounding truth* de $\mathbf{X}_{t+1}$. Esta era la naturaleza autoregresiva de nuestros modelos.

En la siguiente figura mostramos el proceso de cálculo nuestra capa recurrente

![An RNN with a hidden state.](http://d2l.ai/_images/rnn.svg)

En la figura, vemos que ocurre a cada instante $t$:

1. Tenemos una capa densa con función de activación $\phi$ que toma nuestra matriz de diseño $\mathbf{X}_t$ y nuestra variable oculta $\mathbf{H}_{t-1}$.
2. A la salida generamos nuestra nueva variable oculta $\mathbf{H}_t$.
3. Con $\mathbf{H}_t$ y otra capa densa generamos nuestra salida $\mathbf{O}_t$

Muchas veces, en el paso 1 lo que se hace es concatenar $\mathbf{X}_t$ y $\mathbf{H}_{t-1}$ para de esa manera definir una unica matriz de pesos. A continuación mostramos como esta concatenación genera el mismo resultado.

In [ ]:
import torch

In [ ]:
X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 0.6495, -0.0479, -1.5496,  3.3693],
        [-2.7558, -0.9881, -0.2930,  4.0410],
        [ 0.0106,  0.6291,  0.8908,  0.0216]])

In [ ]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[ 0.6495, -0.0479, -1.5496,  3.3693],
        [-2.7558, -0.9881, -0.2930,  4.0410],
        [ 0.0106,  0.6291,  0.8908,  0.0216]])

## Problemas con nuestras predicciones 

Un último punto que hemos evitado hasta ahora es el problema de la predicción en secuencias temporales. Dijimos más arriba que la salida $\mathbf{O}_t$ debe coincidir con el resultado correcto o *grounding truth* de $\mathbf{X}_{t+1}$. Sin embargo, para entrenar correctamente nuestro modelo debemos cada tanto usar la salida $\mathbf{O}_t$ en lugar de $\mathbf{X}_{t+1}$. Si usamos siempre nuestra *grounding truth* en lugar de nuestra predicción, corremos el riesgo de que nuestra red no aprenda a adaptarse a malas predicciones que genera. Al mismo tiempo, nuestra red aprende a modelar las secuencia de entrenamiento, pero puede no saber que hacer con oraciones nuevas.

El ejemplo sería como el siguiente:

Entrenamos una red con "El ingenioso hidalgo Don Quijote de la Mancha". Luego usamos la red para completar:

> *En un lugar de la Mancha de cuyo nombre prefiero no ...*

La red predice "recordar", en lugar de "acordarme". En el siguiente paso, si usamos "acordarme" la nueva predicción podría ser "vivía", pero si usaramos "recordar" la predicción podría ser "Residía". Es decir, al usar solo la *grounding truth* en lugar de nuestra predicción, el modelo no aprende a corregir sus errores, y se queda "estancado" en los datos de entrenamiento. El mayor problema es que cuando tengamos un modelo funcionando, NO HAY *GROUNDING TRUTH*. Esto hace que nuestro modelo pueda fallar estrepitosamente si no usamos nuestra predicción como entrada al modelo. Pero al mismo tiempo, si usamos siempre nuestra predicción, los errores se acumular paulatinamente.

Más adelante hablaremos de la técnica llamada **teacher forcing** y como elegir cuando usar la predicción y el *grounding truth* para generar redes que puedan adaptarse a la variabilidad de nuestra predicción.

# Preliminares a la implementación de RNN

Antes de discutir pasar a implementar una RNN desde cero, queremos discutir algunos temás más que serán importantes conocer. 



## Muestro de secuencias.

Cuando tenemos que elegir que ejemplos debemos usar de un dataset que no contiene secuencias, simplemente mezclabamos aleatoriamente los ejemplos y luego los usabamos para entrenar nuestras redes. 

Sin embargo, en secuencias temporales no podemos hacer esto. Si mezclamos aleatriamente podemos terminar generando secuencias sin sentido

> *En un lugar de la Mancha de cuyo nombre prefiero no acordarme*

luego de mezclarlo

> *nombre En de de no lugar prefiero la Mancha cuyo acordarme un*

Por esta razón debemos generar paritiones y mezclarlas. Por ejemplo podemos generar particiones de 4 elementos 

> [*En un lugar de*] [*la Mancha de cuyo*] [*nombre prefiero no acordarme*]

> [*la Mancha de cuyo*][*nombre prefiero no acordarme*] [*En un lugar de*]

Además de, podemos elegir un offset o deplazamiento. En el ejemplo anterior, un offset de 1 generaría: 

> *En* [*un lugar de la*] [*Mancha de cuyo nombre*] [*prefiero no acordarme, no*]



## Perplejidad

La perplejidad es una métrica que es usada en procesamiento de lenguajes naturales para tener una idea de que tan "convencido" está  nuestro modelo de la siguiente palabra que adivinará. Como métrica está relacionada a la entropía y la entropía cruzada, por lo que volveremos a usar los ejemplos del juego que discutimos anteriormente.

Recordemos nuestro juego:

* Materiales:
  * una bolsa o recipiente opaco
  * 4 pelotas con los números 1, 2, 3, 4
* Preparativos:
  * Se colocan las pelotas en la bolsa 
  * El primer jugador saca una de las pelotas de la bolsa
* Objetivo general: Adivinar con el menor número de preguntas posibles cuál el número de la pelota que tiene el primer jugador .
  * Solo pueden hacerse preguntas que tengan como respuestas sí o no.

Recordemos la estrategia optima:

```
1. Preguntar: "¿El número es par?"
  A. Si la respuesta es sí, preguntar: "¿Es el número 4?"
    a. Si la respuesta es sí, sabemos que es el número 4, hemos ganado.
    b. Si la respuesta es no, sabemos que es el número 2, hemos ganado.
  B. Si la respuesta es no, preguntar: "¿Es el número 3?"
    a. Si la respuesta es sí, sabemos que es el número 3, hemos ganado.
    b. Si la respuesta es no, sabemos que es el número 1, hemos ganado.
```

Recordemos la entropía de nuestra estrategia:

||$1$|$2$|$3$|$4$|total|
|---|---|---|---|---|:-:|
|probabilidad de ocurrir|$\dfrac{1}{4}$|$\dfrac{1}{4}$|$\dfrac{1}{4}$|$\dfrac{1}{4}$|-|
|número de preguntas|$2$|$2$|$2$|$2$|-|
|producto|$\dfrac{1}{2}$|$\dfrac{1}{2}$|$\dfrac{1}{2}$|$\dfrac{1}{2}$|$2$|

Ahora preguntamos, ¿cuantas opciones posibles tenemos en nuestro juego? 
> Como todas las pelotas son equiprobables, tenemos 4 opciones distintas.

La pregunta ahora es que pasará en nuestros segundo juego cuando preguntemos ¿cuantas opciones posibles hay? 

Segundo juego:

* Materiales:
  * una bolsa o recipiente opaco
  * 8 pelotas con los números 1, 1, 1, 1, 2, 2, 3, 4
* Preparativos:
  * Se colocan las pelotas en la bolsa 
  * El primer jugador saca una de las pelotas de la bolsa
* Objetivo general: Adivinar con el menor número de preguntas posibles cuál el número de la pelota que tiene el primer jugador .
  * Solo pueden hacerse preguntas que tengan como respuestas sí o no.

Estrategia optima

```
1. Preguntar: "¿Es el número 1?"
  A. Si la respuesta es sí, hemos ganado."
  B. Si la respuesta es no, preguntar: "¿Es el número 2?"
    a. Si la respuesta es sí, hemos ganado.
    b. Si la respuesta es no, preguntar: "¿Es el número 3?.
      I. Si la respuesta es sí, sabemos que es el número 3, hemos ganado.
      I. Si la respuesta es no, sabemos que es el número 4, hemos ganado.
```

||$1$|$2$|$3$|$4$|total|
|---|---|---|---|---|:-:|
|probabilidad de ocurrir|$\dfrac{4}{8}$|$\dfrac{2}{8}$|$\dfrac{1}{8}$|$\dfrac{1}{8}$|-|
|número de preguntas|$1$|$2$|$3$|$3$|-|
|producto|$\dfrac{1}{2}$|$\dfrac{1}{2}$|$\dfrac{3}{8}$|$\dfrac{3}{8}$|$1.75$|

Dado que la probabilidad de la pelota 1 es mucho mayor que las demás, no tiene sentido decir que tenemos 4 opciones equiprobables. Por el contrario, debemos tener menos de 2 opciones. Esto es porque el 75% de las veces caeremos en la pelota con el número 1 o la pelota con el número 2. La pregunta es, como encontramos esta cantidad de opciones en promedio. 



En el primer juego, la cantidad de opciones promedio era 4 y la entropia era 2. Si modificamos el primer juego usando 8 pelotas distintas que puedan ocurrir de manera equiprobable, veíamos que la entropía es 3, y el número de opciones 8. Facilmente podemos ver como sera el resultado final.

Si tenemos $n$ eventos equiprobables, la entropía sera:

$$P(x) = \dfrac{1}{n},∀x\in\mathbb{Z}~\land 1\leqslant x \leqslant n $$

$$H(P(x)) = \log_2(n)$$

Del resultado anterior podemos ver que el número de opciones equiprobables $n$ es igual a $2^H$. Esta cantidad, la llamaremos perplejidad y la usaremos para definir el número de opciones promedio para una distribución probabilistica arbitraria. 

$$\text{PPL}(P(x)) = 2^{H(P(X))}$$

Para nuestro segun juego tenemos que su perplejidad es de $2^{1.75} ≈3.364$

Rápidamente podemos ver que la perplejidad nos permite una interpretación del número. Una perplejidad igual a 1, indica que nuestro modelo está convencido de cual será la siguiente palabra. Mientras que si nuestro modelo está entrenado con 10000 palabras y tiene una perplejidad de 10000, nuestro modelo nunca sabe cual de las palabras poner a continuación.

De la misma manera, definimos una perplejidad para una entroía cruzada. Sin embargo, para la entropía cruzada la perplejidad puede dar infinito. Esto sería equivalente a decir que nuestro modelo aprendió algo que  nada tiene que ver con el problema que queríamos resolver. Está tan perdido que no sabe que hacer. Algo parecido a lo que le puede pasar a un hablante de español cuando llega a una país con una lengua que desconoce como puede ser el neerlandés.



# Implementando una RNN desde 0

Ahora sí, implmentaremos una RNN desde 0



In [ ]:
class RNNScratch(torch.nn.Module):
    def __init__(self, num_inputs, num_hiddens):
        super().__init__()
        self.num_hiddens = num_hiddens
        self.num_inputs = num_inputs
        self.W_xh = torch.nn.Parameter(
            torch.randn(num_inputs, num_hiddens) * 0.01)
        self.W_hh = torch.nn.Parameter(
            torch.randn(num_hiddens, num_hiddens) * 0.01)
        self.b_h = torch.nn.Parameter(torch.zeros(num_hiddens))

    def forward(self, inputs, state=None):
        if state is not None:
            state, = state
        outputs = []
        for X in inputs:  # Shape of inputs: (num_steps, batch_size, num_inputs)
            state = torch.tanh(torch.matmul(X, self.W_xh) + (
                torch.matmul(state, self.W_hh) if state is not None else 0)
                             + self.b_h)
            outputs.append(state)
        return outputs, state

Probemos que ocurre al generar una RNN y luego alimentarla con un tensor arbitrario

In [ ]:
# 5 secuencias de longitud 3 de vectores de 4 componentes
A = torch.randn(5, 3, 4)
print(A[0]) # una secuencia de 3 vectores de 4 componentes

# red recurrente que toma vectores de 4 componentes y entrega vectores de 2
rnn =  RNNScratch(4, 2)

O, H = rnn(A)

print(len(O), O[0].shape) # 5 secuencias de longitud 3 de vectores de 2 componentes
print(H) # 3 vectores de 2 componentes

tensor([[ 0.2819, -0.2242,  1.5291, -1.4235],
        [ 1.3052,  0.0299,  1.3173,  0.6242],
        [ 0.8394,  0.3529, -0.3716,  1.9146]])
5 torch.Size([3, 2])
tensor([[-0.0339,  0.0065],
        [ 0.0002, -0.0324],
        [-0.0014, -0.0100]], grad_fn=<TanhBackward0>)


Hay que considerar que hasta ahora solo hemos creado la variable oculta $\mathbf{H}$, no hemos aplicado la capa densa final que debemos usar para predecir la próxima palabra.

In [ ]:
class RNNLMScratch(torch.nn.Module):
    """Defined in :numref:`sec_rnn-scratch`"""
    def __init__(self, rnn, vocab_size):
        super().__init__()
        self.rnn = rnn
        self.vocab_size = vocab_size
        self.init_params()

    def init_params(self):
        self.W_hq = torch.nn.Parameter(
            torch.randn(
                self.rnn.num_hiddens, self.vocab_size) * 0.01)
        self.b_q = torch.nn.Parameter(torch.zeros(self.vocab_size))

    def one_hot(self, X):
        """Defined in :numref:`sec_rnn-scratch`"""
        # Output shape: (num_steps, batch_size, vocab_size)
        return  torch.nn.functional.one_hot(X.T,
                                            self.vocab_size).type(torch.float32)

    def output_layer(self, rnn_outputs):
        """Defined in :numref:`sec_rnn-scratch`"""
        outputs = [torch.matmul(H, self.W_hq) + self.b_q for H in rnn_outputs]
        return torch.stack(outputs, 1)
        

    def forward(self, X, state=None):
        """Defined in :numref:`sec_rnn-scratch`"""
        embs = self.one_hot(X)
        rnn_outputs, _ = self.rnn(embs, state)
        return self.output_layer(rnn_outputs)

    def predict(self, prefix, num_preds, vocab, device=None):
        """Defined in :numref:`sec_rnn-scratch`"""
        state, outputs = None, [vocab[prefix[0]]]
        for i in range(len(prefix) + num_preds - 1):
            X = torch.tensor([[outputs[-1]]], device=device)
            embs = self.one_hot(X)
            rnn_outputs, state = self.rnn(embs, state)
            #if type(state) is tuple: state = state[0]
            if i < len(prefix) - 1:  # Warm-up period
                outputs.append(vocab[prefix[i + 1]])
            else:  # Predict `num_preds` steps
                Y = self.output_layer(rnn_outputs)
                outputs.append(int(torch.reshape(torch.argmax(Y, axis=2),
                                                 (1,))))
        return ''.join([vocab.get_itos()[i] for i in outputs])

También aquí podemos revisar con que entramos a nuestra red y con que salimos.

In [ ]:
model = RNNLMScratch(rnn, 4)
outputs = model(torch.ones((3, 5), dtype=torch.int64))
outputs.shape

torch.Size([3, 5, 4])

## Backpropagation en el tiempo

Ahora bien, esta implmentación nos permite señalar el mayor problema que tienen la RNN. Para esto, consideremos el gradiente de la función de pérdida como si solo tuvieramos la variable oculta a la salida.

Como analizamos varias salidas a diferentes instantes, debemos pesar sobre cada uno de nuestros instantes. Esto define una función de pérdida promediada en el tiempo:

$$L(x_1, \ldots, x_T, y_1, \ldots, y_T, w_h, w_o) = \frac{1}{T}\sum_{t=1}^T l(y_t, o_t).$$

Apliquemos a continuación los correspondientes gradientes:

$$\begin{aligned}\frac{\partial L}{\partial w_h}  & = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)}{\partial w_h}  \\& = \frac{1}{T}\sum_{t=1}^T \frac{\partial l(y_t, o_t)}{\partial o_t} \frac{\partial g(h_t, w_o)}{\partial h_t}  \frac{\partial h_t}{\partial w_h}.\end{aligned}$$

En donde hemos usado la definición de las cariables $h_t$ y $o_t$

$$\begin{aligned}h_t &= f(x_t, h_{t-1}, w_h),\\o_t &= g(h_t, w_o),\end{aligned}$$

El problema de las redes recurrentes justamente lo tenemos en el tercer factor. Si calculamos la derivada parcial encontramos que $h_t$ depende de $h_{t-1}$, que a su vez depende de $h_{t-2}$...

$$\frac{\partial h_t}{\partial w_h}= \frac{\partial f(x_{t},h_{t-1},w_h)}{\partial w_h} +\frac{\partial f(x_{t},h_{t-1},w_h)}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial w_h}.$$

Estamos en un problema de recurrencia. Si ha trabajado con este tipo de problemas, sabe que estos pueden dar lugar a recurrencias poco costosas (el cálculo del factorial de $t!$ depende de unas $t$ operaciones) o ridículamente costosas (el cálculo del $t$-ésimo número de Fibonacci depende de alrededor $2^t$ operaciones).

Puede demostrarse que la expresión anterior es análoga al algoritmo de cálculo de un polinomio como los implementados en programas como Mathematica o Matlab o bibliotecas como `numpy`. Se sabe que este cálculo sólo depende de $t$ operaciones... a condicion de que hayamos cuardado todos los gradientes anteriores. Esto último es en realidad nuestro mayor problema: nuestro gradiente a cada instante $t$ es una matriz en el mejor de los casos, con lo cual necesitaremos guardar $t$ matrices para el tiempo $t$, $t+1$ para el tiempo $t+1$... O en su defecto volver a calcular cada gradiente desde cero.

Este problema en donde necesitamos memoria de trabajo infinita o tener que volver a calcular multiples matrices es un problema de alguna manera insalvable para el cálculo del gradiente. Por este motivo, las soluciones consisten es hacer una aproximación. Las aproximaciones son dos:

* Detener el cálculo del gradiente más alla de un número fijo de pasos. Es decir, decidir que le cálculo del gradiente recursivo se detendra luego de 10 pasos hacia atras. 10 es un número arbitrario que elegimos segun conveniencia
* Detener el cálculo de manera aleatoria. Antes de calcular cada paso hacia atras, una distribución probabilistica no dice si debemos tenernos o no. Además, debemos hacer algunas correciones para que el valor esperado de nuestro gradiente calculado de manera aleatoria, coincida con el valor real. 

Se ha visto que estas dos soluciones producen resultados similares y que ninguno consituye una gran mejora respecto al otro.

## *Gradient clipping*

Dijimos anteriormente que la relación de recurrencia anterior equivale al cálculo de una polinomio de grado $t$. Pues bien, eso significa que para el tiempo $t$ tenemos un término que depende aproximadamente de la $t$-ésima potencia del gradiente de las variables ocultas. Dado que este gradiente es una matriz, estamos hablando una matriz elevada a la potencia $t$. El resultado es que los valores de nuestra matriz pueden crecer demasiado. Es por esto que para evitar un crecimiento descontrolado de nuestro gradiente, se realiza lo que se conoce como *gradient clipping*. Es decir, cuando el gradiente es mayor a cierta cantidad se procede a entregar un valor fijo de gradiente por encima del umbral definido. En efecto, lo que se hace es calcular el gradiente de la siguiente manera:

$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$

De esta manera el modulo del gradiente nunca supera la cantidad $\theta$

In [ ]:
def clip_gradients(grad_clip_val, model):
        params = [p for p in model.parameters() if p.requires_grad]
        norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
        if norm > grad_clip_val:
            for param in params:
                param.grad[:] *= grad_clip_val / norm

# Preprocesamiento

Recordemos un momento como es nuestro pipeline:

1. Carga de los datos
1. Separación de los datos en lotes
1. Inicialización de parámetros 
1. Definición del modelo
1. Definición de la función de pérdida
1. Definición del algoritmo de optimización

En líneas generales hemos presentado todos los pasos de nuestro pipeline, pero debemos tener cuidado con el proceso de tokenización, como se explicó anteriormente. Por eso presentaremos algunas herramientas muy sencillas de tokenización.

Nuestra tarea, en este caso, sera tratar de enseñarle a nuestra red a escribir correctamente en español letra por letra. Para esto hemos elegido "El ingenioso hidalgo Don Quijote de la Mancha" como texto de referencia. Usaramos la letras del mismo texto para enseñarle a nuestro modelo a escribir palabras en español. Para eso tokenizaremos las letras del español.

In [ ]:
import re
import collections
import torchtext

def make_vocab(fn, skip=0):
  data = None
  with open(fn, "r") as f: 
    f.seek(skip)
    data = f.read()
  if data == None:
    return None, None
    # ".." match " "
    # ".;" match " "
    # ".a" match " a"
    # " . " match "." reemplaza "   "
    # " .. " match ".." reemplaza "   "
    # guía no machea nada "guía"
  tokens = re.sub('[^A-Za-záéíóúÁÉÍÓÚñÑüÜ]+', ' ', data).lower()
  tokens = [token for line in list(tokens) for token in line]
  counter = collections.Counter(tokens)
  freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
  ordered_dict = collections.OrderedDict(freq_tuples)
  result = torchtext.vocab.vocab(ordered_dict)
  corpus = [result[token] for token in tokens]
  return result, ordered_dict, corpus

!wget https://www.gutenberg.org/files/2000/2000-0.txt
vocab, dictionary, corpus = make_vocab("2000-0.txt", 41508)

--2022-08-17 15:21:45--  https://www.gutenberg.org/files/2000/2000-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2226045 (2.1M) [text/plain]
Saving to: ‘2000-0.txt.1’

2000-0.txt.1        100%[===================>]   2.12M  --.-KB/s    in 0.1s    

2022-08-17 15:21:45 (19.6 MB/s) - ‘2000-0.txt.1’ saved [2226045/2226045]



In [ ]:
batch_size = 1024
num_steps = 32
array = torch.tensor([corpus[i:i+num_steps+1]
                            for i in range(0, len(corpus)-num_steps-1)])
# qwert y (i = 0)
# q werty (i = 1)
features, tags = array[:,:-1], array[:,1:]

num_train = 20480 
num_val = 5120 
def get_tensorloader(tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, batch_size,
                                        shuffle=train)

train_iter = get_tensorloader([features, tags], True, indices=slice(0, num_train))
test_iter = get_tensorloader([features, tags], False,
                             indices=slice(num_train, num_train + num_val))


In [ ]:
print(vocab[" "])
print(vocab["e"])
print(vocab["a"])
print(vocab["á"])
print(vocab["ñ"])
print(type(vocab.get_itos()) is list)
print(vocab.get_itos())
print(vocab.get_itos()[9])

0
1
2
26
28
True
[' ', 'e', 'a', 'o', 's', 'n', 'r', 'l', 'd', 'u', 'i', 't', 'c', 'm', 'p', 'q', 'y', 'b', 'h', 'v', 'g', 'í', 'j', 'ó', 'f', 'é', 'á', 'z', 'ñ', 'ú', 'x', 'w', 'k', 'ü']
u


Haremos unas pequeña redifinición a nuestra función de pérdida, dado que estamos tamos trabajando con tensores con 3 dimensiones

In [ ]:
def loss_NLP(Y_hat, Y):
    Y_hat = torch.reshape(Y_hat, (-1, Y_hat.shape[-1]))
    Y = torch.reshape(Y, (-1,))
    return torch.nn.functional.cross_entropy(
        Y_hat, Y, reduction='none')

# Entrenamiento

In [ ]:
rnn_scrt =  RNNScratch(num_inputs=len(vocab), num_hiddens=32)
net1 = RNNLMScratch(rnn_scrt, vocab_size=len(vocab))
loss = loss_NLP
trainer = torch.optim.Adadelta(net1.parameters(), lr=1)

num_epochs = 100
for epoch in range(num_epochs):
    L = 0.0
    N = 0
    TestN = 0
    TestL = 0
    for X, Y in train_iter:
        l = loss(net1(X), Y)
        trainer.zero_grad()
        l.mean().backward()
        clip_gradients(grad_clip_val = 1, model = net1)
        trainer.step()
        L += l.sum()
        N += l.numel()
    for X, Y in test_iter:
        TestL += l.sum()
        TestN += Y.numel()
    print(f'epoch {epoch + 1}')
    print(f'    loss {float(L/N):f},')
    print(f'    train perplexity {torch.exp((L/N)):f},')
    print(f'    test perplexity {torch.exp((TestL/TestN)):f}.')
    print()




epoch 1
    loss 3.480818,
    train perplexity 32.486294,
    test perplexity 29.455894.

epoch 2
    loss 2.951564,
    train perplexity 19.135860,
    test perplexity 17.336414.

epoch 3
    loss 2.847047,
    train perplexity 17.236809,
    test perplexity 17.144173.

epoch 4
    loss 2.831578,
    train perplexity 16.972218,
    test perplexity 16.727331.

epoch 5
    loss 2.797822,
    train perplexity 16.408869,
    test perplexity 15.961906.

epoch 6
    loss 2.739798,
    train perplexity 15.483862,
    test perplexity 14.872507.

epoch 7
    loss 2.641134,
    train perplexity 14.029107,
    test perplexity 13.375977.

epoch 8
    loss 2.551752,
    train perplexity 12.829556,
    test perplexity 12.329431.

epoch 9
    loss 2.480552,
    train perplexity 11.947857,
    test perplexity 11.563040.

epoch 10
    loss 2.421227,
    train perplexity 11.259661,
    test perplexity 11.026072.

epoch 11
    loss 2.378724,
    train perplexity 10.791121,
    test perplexity 10.488839

Veamos el resultado final.

In [ ]:
net1.predict("quijote y sancho ", 40, vocab)

'quijote y sancho de la mantro de la mante de la mante de '

# Implementación concisa de RNN

En función a los dos problemas asociados a Backpropagation en el tiempo y al crecieminto descontrolado de los gradientes, vemos que es preferible usar las herramientas que ya trae `torch`. Para llamar a una RNN que entrega estados ocultos deberemos hacer lo que se muestra en el siguiente código

In [ ]:
class RNN(torch.nn.Module):
    def __init__(self, num_inputs, num_hiddens):
        super().__init__()
        self.rnn = torch.nn.RNN(num_inputs, num_hiddens)

    def forward(self, inputs, H=None):
        return self.rnn(inputs, H)

In [ ]:
class RNNLM(RNNLMScratch):
    def init_params(self):
        self.linear = torch.nn.LazyLinear(self.vocab_size)
    def output_layer(self, hiddens):
        return self.linear(hiddens).swapaxes(0, 1)

In [ ]:
rnn =  RNN(num_inputs=len(vocab), num_hiddens=32)
net2 = RNNLM(rnn, vocab_size=len(vocab))
loss = loss_NLP
trainer = torch.optim.Adadelta(net2.parameters(), lr=1)

num_epochs = 100
for epoch in range(num_epochs):
    L = 0.0
    N = 0
    TestN = 0
    TestL = 0
    for X, Y in train_iter:
        l = loss(net2(X), Y)
        trainer.zero_grad()
        l.mean().backward()
        clip_gradients(grad_clip_val = 1, model = net2)
        trainer.step()
        L += l.sum()
        N += l.numel()
    for X, Y in test_iter:
        TestL += l.sum()
        TestN += Y.numel()
    print(f'epoch {epoch + 1}')
    print(f'    loss {float(L/N):f},')
    print(f'    train perplexity {torch.exp((L/N)):f},')
    print(f'    test perplexity {torch.exp((TestL/TestN)):f}.')
    print()




/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


epoch 1
    loss 3.119849,
    train perplexity 22.642954,
    test perplexity 17.419270.

epoch 2
    loss 2.822018,
    train perplexity 16.810736,
    test perplexity 16.244394.

epoch 3
    loss 2.732061,
    train perplexity 15.364519,
    test perplexity 14.569115.

epoch 4
    loss 2.607428,
    train perplexity 13.564120,
    test perplexity 12.738925.

epoch 5
    loss 2.500514,
    train perplexity 12.188758,
    test perplexity 11.661670.

epoch 6
    loss 2.417577,
    train perplexity 11.218647,
    test perplexity 10.803309.

epoch 7
    loss 2.353830,
    train perplexity 10.525803,
    test perplexity 10.169787.

epoch 8
    loss 2.307568,
    train perplexity 10.049956,
    test perplexity 9.867966.

epoch 9
    loss 2.270337,
    train perplexity 9.682662,
    test perplexity 9.505201.

epoch 10
    loss 2.241402,
    train perplexity 9.406511,
    test perplexity 9.178358.

epoch 11
    loss 2.213364,
    train perplexity 9.146437,
    test perplexity 8.978465.

epoc

In [ ]:
net2.predict("quijote y sancho ", 30, vocab)

'quijote y sancho de la mando de la mando de la '

# Long Short-Term Memory (LSTM)

Uno de los problemas que vimos que tenían las redes recurrentes es las características de sus gradientes hacían que estos pudieran, o bien crecer de manera descontrolada, o bien achicarse hasta 0. En cualquiera de los dos casos nuestra propuesta de solución fue restringir el número de pasos hacia atras en el tiempo en los que calcularemos el gradiente. Sin embargo, este camino puede ser un problema en algunas aplicaciones.

Para esto surgió una alternativa a una RNN, que es la aquitectura LSTM

## Celdas de Memoria

LSTM es una red recurrente: la nueva salida depende de las entradas anteriores. Sin embargo, agrega un conjunto de variables ocultas para intentar emular la memoria RAM de una PC.

Una memoria RAM guarda diferente información para utilizarla luego en un cálculo. Además puede realizar un conjunto de operaciones por ejemplo:

* Leer los valores guardados anteriormente
* Escribir el valor guardado por uno nuevo 
* Borrar lo que había en memoria.

En este sentido LSTM tendrá dos variables ocultas. La primera es nuestra varaible oculta convencional $\mathbf{H}_{t-1}$. Pero la segunda es una varaible $\mathbf{C}_{t-1}$ que guarda información como una memoria RAM. Luego usaremos es información para generar una nueva variable $\mathbf{H}_{t}$ en el próximo paso temporal.

En sintonía con lo anteior necesitaremos señales lógicas que nos ayudaran a decidir que hacer con la nueva entrada $\mathbf{X}_t$ y la varaible oculta anterior $\mathbf{H}_{t-1}$:

* Leer el el valor de memoria $\mathbf{C}_{t-1}$ para calcular $\mathbf{H}_{t}$
* Modificar el valor anterior de $\mathbf{C}_{t-1}$, para crear uno nuevo $\mathbf{H}_t$
* Borrar completamente la memoria $\mathbf{C}_{t} = 0$$

En una memoria RAM real, estás señales con manejadas por señales binarias. Entonces si quisieramos borrar, pondríamos un 1 en entrada de la RAM que recibe la instrucción de borrado. Si quisieramos leer, pondríamos un 0 en la entrada de borrado y un 1 en lade leer, etc.

Al estar trabajando con tensores y problemas de optimización, ahora nuestra salida puede ser continua. es decir, ya no solo podríamos borrar, sino tambien borrar parcialmente. Sin embargo, para esto debemos asegurarnos que nuestras salidas sean valores entre 0 y 1. Para esto crearemos una capa RNN con una sigmoidea como función de activación. Presentemos entonces las primeras 3 compuertas lógicas de nuestro LSTM:

### Compuestas Lógicas


![](http://d2l.ai/_images/lstm-0.svg)

$$
\begin{aligned}
\mathbf{O}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xo} + \mathbf{H}_{t-1} \mathbf{W}_{ho} + \mathbf{b}_o),\\
\mathbf{I}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xi} + \mathbf{H}_{t-1} \mathbf{W}_{hi} + \mathbf{b}_i),\\
\mathbf{F}_t &= \sigma(\mathbf{X}_t \mathbf{W}_{xf} + \mathbf{H}_{t-1} \mathbf{W}_{hf} + \mathbf{b}_f)
\end{aligned}
$$

Es decir, tenemos 3 capas RNN, con sus respectivos pesos. En donde: 

* $\mathbf{O}_{t}$ corresponde a la señal de lectura. Nos dice que tanta importacia debemos darle a los valores anteriores de nuestra variable oculta $\mathbf{H}_{t-1}$
* $\mathbf{I}_{t}$ corresponde a la señal de escritura.Nos dice que tanto debe cambiar nuestra variable oculta anterior $\mathbf{H}_{t-1}$. 
* $\mathbf{F}_{t}$ corresponde a la señal de borrado.Nos dice que tanto eliminar nuestra variable oculta anterior $\mathbf{H}_{t-1}$.

La analogía con el lenguaje es más o menos directa:

* En un libro de botánica, al describir un árbol hay una serie de oraciones relacionadas entre ellas. Nuestra red aprender a mantener la coherencia. Si hablamos de la fruta roja del árbol, no puede luego hablar de la fruta amarilla. Debemos MANTENER el tema de la conversación.
* En una obra de teatro, un personaje puede cambiar su estado. Puede pasar de estar parado a desmayarse. En ese sentido debemos poder MODIFICAR la situación del personajes. De otra manera, sería dificil de entender poque alguien se levanto si nunca dejo de estar parado.
* En una novela, se suceden una serie de acciones, pero no siempre están conectadas entre ellas. Si un capitulo esta centrado en el protagonista y el sigueinte centrado en el villano, debemos OLVIDAR el contexto anterior para no perder el hilo. Si antes el protagonista usaba patalones azules, debemos ignorar eso cuando los pantalones del villano son negros.

Rercordemos una vez más que hemos elegido una función de activación sigmoidea en analogía a las señales de las compuertas lógicas de una memoria RAM.

### Candidato de memoria

Ahora, lo que haremos será calcular el nuevo valor que guardaremos en memoria. Para esto simplementa aplicamos una RNN convencional con una $\tanh$ como función de activación.

$$\tilde{\mathbf{C}}_t = \text{tanh}(\mathbf{X}_t \mathbf{W}_{xc} + \mathbf{H}_{t-1} \mathbf{W}_{hc} + \mathbf{b}_c),$$

Este valor es un valor tentativo con el cual cambiaremos el valor existente en nuestra memoria $\mathbf{C}_t$ 

![](http://d2l.ai/_images/lstm-1.svg)

### Escribiendo en memoria.

Ahora lo que haremos será modificar nuestro valor en memoria. Hay dos operaciones que pueden modificar nuestra memoria: borrado y escritura. Con lo cual haremos una combinación lineal de las dos cosas 

$$\mathbf{C}_t = \mathbf{F}_t \odot \mathbf{C}_{t-1} + \mathbf{I}_t \odot \tilde{\mathbf{C}}_t.$$

En donde hemos usado $\odot$ para representar el **temido** producto de Haddamar. Analicemos con un ejemplo sencillo como calcular el producto de Haddamar de dos matrices:


$$
\mathbf{A} = \begin{bmatrix}2&3\\5&7\end{bmatrix},
\mathbf{B} = \begin{bmatrix}3&5\\7&2\end{bmatrix},\\
\mathbf{A} \odot \mathbf{B} = \begin{bmatrix}2&3\\5&7\end{bmatrix} ⊙ \begin{bmatrix}3&5\\7&2\end{bmatrix}=\begin{bmatrix}6&15\\35&14\end{bmatrix}\\
\mathbf{A} \odot \mathbf{A} = \begin{bmatrix}2&3\\5&7\end{bmatrix} ⊙ \begin{bmatrix}2&3\\5&7\end{bmatrix}=\begin{bmatrix}4&9\\25&49\end{bmatrix}
$$

Vemos que el producto de Haddamar es en esencia no es más que multiplicar elemento a elemento de una matriz o un tensor. Es simplemente un nombre raro, para algo que es mucho más intuitivo que la multiplicación de matrices tradicionales.

![](http://d2l.ai/_images/lstm-2.svg)

Con viene analizar que ocurre en cada caso para $\mathbf{F}_{t}$ y $\mathbf{I}_{t}$

||$\mathbf{I}_{t} = 1$| $\mathbf{I}_{t} = 0$
|---|:-:|:-:|
|$\mathbf{F}_{t}=1$|Combinación lineal del valor nuevo y el viejo|Se conserva el valor viejo|
|$\mathbf{F}_{t}=0$|Se reemplaza el valor nuevo por el viejo|Se borra la celda de memoria|

### Estado oculto.

Ahora sí, leeremos la memoria para obtener nuestro nueva variable oculta

$$\mathbf{H}_t = \mathbf{O}_t \odot \tanh(\mathbf{C}_t).$$

Es decir, aplicamos una última transformación a nuestro valor de memoria y luego decidimos cuanto leeremos de ese valor. Si $\mathbf{O}_t = 0$, ignoraremos lo que haya en memoria, pero si $\mathbf{O}_t = 1$, le prestaremos total antención.

![](http://d2l.ai/_images/lstm-3.svg)


## Implementation de LSTM desde 0


In [ ]:
class LSTMScratch(torch.nn.Module):
    def __init__(self, num_inputs, num_hiddens):
        super().__init__()

        init_weight = lambda *shape: torch.nn.Parameter(torch.randn(*shape) * 0.01)
        triple = lambda: (init_weight(num_inputs, num_hiddens),
                          init_weight(num_hiddens, num_hiddens),
                          torch.nn.Parameter(torch.zeros(num_hiddens)))
        self.num_hiddens = num_hiddens
        self.num_inputs = num_inputs
        self.W_xi, self.W_hi, self.b_i = triple()  # Input gate
        self.W_xf, self.W_hf, self.b_f = triple()  # Forget gate
        self.W_xo, self.W_ho, self.b_o = triple()  # Output gate
        self.W_xc, self.W_hc, self.b_c = triple()  # Candidate memory cell

    def forward(self, inputs, H_C=None):
        H, C = None, None if H_C is None else H_C
        outputs = []
        for X in inputs:
            I = torch.sigmoid(torch.matmul(X, self.W_xi) + (
                torch.matmul(H, self.W_hi) if H is not None else 0) + self.b_i)
            if H is None:
                H, C = torch.zeros_like(I), torch.zeros_like(I)
            F = torch.sigmoid(torch.matmul(X, self.W_xf) +
                            torch.matmul(H, self.W_hf) + self.b_f)
            O = torch.sigmoid(torch.matmul(X, self.W_xo) +
                            torch.matmul(H, self.W_ho) + self.b_o)
            C_tilda = torch.tanh(torch.matmul(X, self.W_xc) +
                              torch.matmul(H, self.W_hc) + self.b_c)
            C = F * C + I * C_tilda # El prod. de Haddar es el producto normal!
            H = O * torch.tanh(C)
            outputs.append(H)
        return outputs, (H, C)

### Entrenamiento




In [ ]:
lstm_scrt = LSTMScratch(num_inputs=len(vocab), num_hiddens=32)
net3 = RNNLMScratch(lstm_scrt, vocab_size=len(vocab))
trainer = torch.optim.Adadelta(net3.parameters(), lr=4)
loss = loss_NLP

num_epochs = 100 
for epoch in range(num_epochs):
    L = 0.0
    N = 0
    TestN = 0
    TestL = 0
    for X, Y in train_iter:
        l = loss(net3(X), Y)
        trainer.zero_grad()
        l.mean().backward()
        clip_gradients(grad_clip_val = 1, model = net3)
        trainer.step()
        L += l.sum()
        N += l.numel()
    for X, Y in test_iter:
        TestL += l.sum()
        TestN += Y.numel()
    print(f'epoch {epoch + 1}')
    print(f'    loss {float(L/N):f},')
    print(f'    train perplexity {torch.exp((L/N)):f},')
    print(f'    test perplexity {torch.exp((TestL/TestN)):f}.')
    print()


epoch 1
    loss 3.287472,
    train perplexity 26.775084,
    test perplexity 17.895092.

epoch 2
    loss 2.856624,
    train perplexity 17.402679,
    test perplexity 17.120613.

epoch 3
    loss 2.839097,
    train perplexity 17.100321,
    test perplexity 16.994011.

epoch 4
    loss 2.822800,
    train perplexity 16.823891,
    test perplexity 16.699621.

epoch 5
    loss 2.795369,
    train perplexity 16.368662,
    test perplexity 16.146240.

epoch 6
    loss 2.750924,
    train perplexity 15.657090,
    test perplexity 15.113146.

epoch 7
    loss 2.656917,
    train perplexity 14.252276,
    test perplexity 13.472737.

epoch 8
    loss 2.539634,
    train perplexity 12.675031,
    test perplexity 12.074724.

epoch 9
    loss 2.438794,
    train perplexity 11.459208,
    test perplexity 11.034871.

epoch 10
    loss 2.369293,
    train perplexity 10.689837,
    test perplexity 10.446251.

epoch 11
    loss 2.315131,
    train perplexity 10.126254,
    test perplexity 9.908369.

In [ ]:
net3.predict("sancho y quijote", 30, vocab)

'sancho y quijote de de de de de de de de de de'

## Implementación Concisa


In [ ]:
class LSTM(RNN):
    def __init__(self, num_inputs, num_hiddens):
        torch.nn.Module.__init__(self)
        self.rnn = torch.nn.LSTM(num_inputs, num_hiddens)

    def forward(self, inputs, H_C=None):
        return self.rnn(inputs, H_C)

In [ ]:
lstm = LSTM(num_inputs=len(vocab), num_hiddens=32)
net4 = RNNLM(lstm, vocab_size=len(vocab))
trainer = torch.optim.Adadelta(net4.parameters(), lr=4)
loss = loss_NLP

num_epochs = 100
for epoch in range(num_epochs):
    L = 0.0
    N = 0
    TestN = 0
    TestL = 0
    for X, Y in train_iter:
        l = loss(net4(X), Y)
        trainer.zero_grad()
        l.mean().backward()
        clip_gradients(grad_clip_val = 1, model = net4)
        trainer.step()
        L += l.sum()
        N += l.numel()
    for X, Y in test_iter:
        TestL += l.sum()
        TestN += Y.numel()
    print(f'epoch {epoch + 1}')
    print(f'    loss {float(L/N):f},')
    print(f'    train perplexity {torch.exp((L/N)):f},')
    print(f'    test perplexity {torch.exp((TestL/TestN)):f}.')
    print()


epoch 1
    loss 3.037571,
    train perplexity 20.854521,
    test perplexity 17.166391.

epoch 2
    loss 2.824945,
    train perplexity 16.860025,
    test perplexity 16.526625.

epoch 3
    loss 2.743249,
    train perplexity 15.537383,
    test perplexity 14.544025.

epoch 4
    loss 2.580240,
    train perplexity 13.200306,
    test perplexity 12.303865.

epoch 5
    loss 2.452065,
    train perplexity 11.612301,
    test perplexity 11.045283.

epoch 6
    loss 2.366464,
    train perplexity 10.659632,
    test perplexity 10.206488.

epoch 7
    loss 2.305007,
    train perplexity 10.024243,
    test perplexity 9.596799.

epoch 8
    loss 2.249058,
    train perplexity 9.478807,
    test perplexity 9.282849.

epoch 9
    loss 2.213953,
    train perplexity 9.151822,
    test perplexity 8.921330.

epoch 10
    loss 2.183291,
    train perplexity 8.875464,
    test perplexity 8.815040.

epoch 11
    loss 2.160307,
    train perplexity 8.673803,
    test perplexity 8.539734.

epoch 

In [ ]:
net4.predict('levantose ', 20, vocab)

'levantose a la mantas de la ma'

# Gated Recurrent Units (GRU)

La arquitectura de LSTM es una arquitectura de la década de 1990. Sin embargo, en 2014, de desarrolló una alternativa más simple que LSTM y que tiene un comportamiento similar. Esta es GRU.

### Compuertas lógicas

Al igual que LSTM, GRU tambien usa unas compuertas lógicas con salidas entre 0 y 1

![Computing the reset gate and the update gate in a GRU model.](https://d2l.ai/_images/gru-1.svg)


$$
\begin{aligned}
\mathbf{R}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xr} + \mathbf{H}_{t-1} \mathbf{W}_{hr} + \mathbf{b}_r),\\
\mathbf{Z}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xz} + \mathbf{H}_{t-1} \mathbf{W}_{hz} + \mathbf{b}_z),
\end{aligned}
$$

### Candidato de variable oculta

Luego, en lugar de calcular el valor de una celda de memoria, GRU directamente propone un nuevo valor de variable oculta.

$$\tilde{\mathbf{H}}_t = \tanh(\mathbf{X}_t \mathbf{W}_{xh} + \left(\mathbf{R}_t \odot \mathbf{H}_{t-1}\right) \mathbf{W}_{hh} + \mathbf{b}_h),$$

En donde, si $\mathbf{R}_t = 0$ ignoramos o *reseteamos* la varaible oculta. Pero si $\mathbf{R}_t =1$ conservamos toda su información.

![](https://d2l.ai/_images/gru-2.svg)

Destacamos que hemos vuelto a usar el producto de Haddamar o producto elemento a elemento.

### Variable oculta

Finalmente usamos nuestra otra compurta lógica para definir que tanta importación le damos al candidato nuevo actual con respecto al valor anteior anterior.

$$\mathbf{H}_t = \mathbf{Z}_t \odot \mathbf{H}_{t-1}  + (1 - \mathbf{Z}_t) \odot \tilde{\mathbf{H}}_t.$$

Es decir, si $\mathbf{Z}_t = 1$  conservamos el valor anteior y no actualizamos nuestra variable. Pero si $\mathbf{Z}_t = 0$, ignoramos el valor anterior y conservamos al candidato.

![](https://d2l.ai/_images/gru-3.svg)





## Implementación de GRU desde 0

In [ ]:
class GRUScratch(torch.nn.Module):
    def __init__(self, num_inputs, num_hiddens):
        super().__init__()

        init_weight = lambda *shape: torch.nn.Parameter(torch.randn(*shape) * 0.01)
        triple = lambda: (init_weight(num_inputs, num_hiddens),
                          init_weight(num_hiddens, num_hiddens),
                          torch.nn.Parameter(torch.zeros(num_hiddens)))
        self.num_hiddens = num_hiddens
        self.num_inputs = num_inputs
        self.W_xz, self.W_hz, self.b_z = triple()  # Update gate
        self.W_xr, self.W_hr, self.b_r = triple()  # Reset gate
        self.W_xh, self.W_hh, self.b_h = triple()  # Candidate hidden state

    def forward(self, inputs, H=None):
        matmul_H = lambda A, B: torch.matmul(A, B) if H is not None else 0
        outputs = []
        for X in inputs:
            Z = torch.sigmoid(torch.matmul(X, self.W_xz) + (
                torch.matmul(H, self.W_hz) if H is not None else 0) + self.b_z)
            if H is None: H = torch.zeros_like(Z)
            R = torch.sigmoid(torch.matmul(X, self.W_xr) +
                            torch.matmul(H, self.W_hr) + self.b_r)
            # R * H es otro producto de Haddamar!!
            H_tilda = torch.tanh(torch.matmul(X, self.W_xh) +
                              torch.matmul(R * H, self.W_hh) + self.b_h)
            H = Z * H + (1 - Z) * H_tilda # Mas prod. de Haddamar!
            outputs.append(H)
        return outputs, H

### Entrenamiento


In [ ]:
gru_scrt = GRUScratch(num_inputs=len(vocab), num_hiddens=32)
net5 = RNNLMScratch(gru_scrt, vocab_size=len(vocab))
trainer = torch.optim.Adadelta(net5.parameters(), lr=4)
loss = loss_NLP

num_epochs = 100
for epoch in range(num_epochs):
    L = 0.0
    N = 0
    TestN = 0
    TestL = 0
    for X, Y in train_iter:
        l = loss(net5(X), Y)
        trainer.zero_grad()
        l.mean().backward()
        clip_gradients(grad_clip_val = 1, model = net5)
        trainer.step()
        L += l.sum()
        N += l.numel()
    for X, Y in test_iter:
        TestL += l.sum()
        TestN += Y.numel()
    print(f'epoch {epoch + 1}')
    print(f'    loss {float(L/N):f},')
    print(f'    train perplexity {torch.exp((L/N)):f},')
    print(f'    test perplexity {torch.exp((TestL/TestN)):f}.')
    print()

epoch 1
    loss 3.153552,
    train perplexity 23.419109,
    test perplexity 17.598305.

epoch 2
    loss 2.843293,
    train perplexity 17.172220,
    test perplexity 16.723484.

epoch 3
    loss 2.763442,
    train perplexity 15.854324,
    test perplexity 14.773314.

epoch 4
    loss 2.590887,
    train perplexity 13.341599,
    test perplexity 12.117282.

epoch 5
    loss 2.421953,
    train perplexity 11.267843,
    test perplexity 10.668628.

epoch 6
    loss 2.328208,
    train perplexity 10.259538,
    test perplexity 9.914872.

epoch 7
    loss 2.261597,
    train perplexity 9.598407,
    test perplexity 9.268740.

epoch 8
    loss 2.211366,
    train perplexity 9.128179,
    test perplexity 8.863169.

epoch 9
    loss 2.170749,
    train perplexity 8.764846,
    test perplexity 8.586558.

epoch 10
    loss 2.138959,
    train perplexity 8.490598,
    test perplexity 8.379647.

epoch 11
    loss 2.114616,
    train perplexity 8.286401,
    test perplexity 8.211603.

epoch 12

In [ ]:
net5.predict("levantose ", 30, vocab)

'levantose en el mucho y a su parecer en '

## Implementación concisa

In [ ]:
class GRU(RNN):
    def __init__(self, num_inputs, num_hiddens):
        torch.nn.Module.__init__(self)
        self.rnn = torch.nn.GRU(num_inputs, num_hiddens)

In [ ]:
gru = GRU(num_inputs=len(vocab), num_hiddens=32)
net6 = RNNLM(gru, vocab_size=len(vocab))
trainer = torch.optim.Adadelta(net6.parameters(), lr=4)
loss = loss_NLP

num_epochs = 100
for epoch in range(num_epochs):
    L = 0.0
    N = 0
    TestN = 0
    TestL = 0
    for X, Y in train_iter:
        l = loss(net6(X), Y)
        trainer.zero_grad()
        l.mean().backward()
        clip_gradients(grad_clip_val = 1, model = net6)
        trainer.step()
        L += l.sum()
        N += l.numel()
    for X, Y in test_iter:
        TestL += l.sum()
        TestN += Y.numel()
    print(f'epoch {epoch + 1}')
    print(f'    loss {float(L/N):f},')
    print(f'    train perplexity {torch.exp((L/N)):f},')
    print(f'    test perplexity {torch.exp((TestL/TestN)):f}.')
    print()

epoch 1
    loss 2.996911,
    train perplexity 20.023594,
    test perplexity 16.715647.

epoch 2
    loss 2.685000,
    train perplexity 14.658208,
    test perplexity 12.532542.

epoch 3
    loss 2.425869,
    train perplexity 11.312056,
    test perplexity 10.439892.

epoch 4
    loss 2.286630,
    train perplexity 9.841719,
    test perplexity 9.436526.

epoch 5
    loss 2.216136,
    train perplexity 9.171826,
    test perplexity 8.849507.

epoch 6
    loss 2.164240,
    train perplexity 8.707982,
    test perplexity 8.512407.

epoch 7
    loss 2.127467,
    train perplexity 8.393582,
    test perplexity 8.233907.

epoch 8
    loss 2.091887,
    train perplexity 8.100182,
    test perplexity 7.954741.

epoch 9
    loss 2.062250,
    train perplexity 7.863642,
    test perplexity 7.702979.

epoch 10
    loss 2.032354,
    train perplexity 7.632034,
    test perplexity 7.624239.

epoch 11
    loss 2.006264,
    train perplexity 7.435490,
    test perplexity 7.454623.

epoch 12
    

In [ ]:
net6.predict('avellaneda ', 20, vocab)

'avellaneda se hallar de la manc'

# Redes bidireccionales y profundas

En general, vamos a ver que muchas veces tener una variable oculta lineal o generada por una sola capa, puede no capturar toda la complejidad de nuestro modelo. Es por esto que debemos tener alguna herramienta que nos permita generar un estado oculto mucho más complejo. Para esto tenemos las redes recurrentes profundas.

En líneas generales, la idea será usar sucesivos redes recurrentes a la salida de nuestra primera capa con estados ocultos. Como muestra la figura

![](https://d2l.ai/_images/deep-rnn.svg)

Afortunadamente este tipo de arquitecturas podemos llamarlas solo agregando un parametro a nuestro código.

In [ ]:
class LSTMDeep(RNN):
    def __init__(self, num_inputs, num_hiddens,num_layers):
        torch.nn.Module.__init__(self)
        self.rnn = torch.nn.LSTM(num_inputs, num_hiddens,num_layers)
        self.num_hiddens = num_hiddens
        self.num_inputs = num_inputs
        self.num_layers = num_layers


    def forward(self, inputs, H_C=None):
        return self.rnn(inputs, H_C)

In [ ]:
lstmD = LSTMDeep(num_inputs=len(vocab), num_hiddens=32, num_layers=2)
net7 = RNNLM(lstm, vocab_size=len(vocab))

Más adelante, veremos que en el problema de traducción, tener información del futuro puede resultar util para los estados presentes. El ejemplo más sencillo es como las palabras se ordenan de manera distinta segun el idioma:

>This is a red pencil

>Este un lápiz es rojo

Por esto también es útil tener los llamados modelos bidireccionales. Estos modelos duplican el número de parametros, pues en esencia entrenan tanto "hacia adelante" temporalmente como "hacia atras"

In [ ]:
class LSTMBidir(RNN):
    def __init__(self, num_inputs, num_hiddens, num_layers, bidirectional):
        torch.nn.Module.__init__(self)
        self.rnn = torch.nn.LSTM(num_inputs,
                                 num_hiddens, num_layers,
                                 bidirectional=bidirectional)
        self.num_hiddens = num_hiddens
        if bidirectional: self.num_hiddens *= 2
        self.num_inputs = num_inputs
        self.num_layers = num_layers

    def forward(self, inputs, H_C=None):
        return self.rnn(inputs, H_C)

In [ ]:
lstmD = LSTMBidir(num_inputs=len(vocab), num_hiddens=32,
                  num_layers=2, bidirectional=True)
net8 = RNNLM(lstm, vocab_size=len(vocab))

Cabe destacar que las redes bidireccionales son un mal modelo para el problema a "aprender a escribir español letra por letra". La razón, de alguna manera es la siguiente:

* Dada una letra "q", ¿cuál es la letra anterior en un texto?
* Dada una letra "q", ¿cuál es la letra siguiente en un texto?

A diferencia de la traducción automática, el deletro tiene mucha más información en la dirección "hacia adelante" que hacia atras.